## Final Project of Big Data

## Import Libraries

In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
#sc = SparkContext('local', 'pyspark') 
sqlContext = SQLContext(sc)

## Load Data Set

In [2]:
loadPath = "../dataset/lending/loan.csv"
dataload = sqlContext.read.csv(loadPath, header='true')
dataload.createOrReplaceTempView("loancredit")
#dataload.cache()

## Show Schema

In [4]:
dataload.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [11]:
dataload.count()

887379

# Clean data

## Check for duplications

In [8]:
print('Count of rows: {0}'.format(dataload.count()))
print('Count of distinct rows: {0}'.format(dataload.distinct().count()))

Count of rows: 887379
Count of distinct rows: 887379


The amount of rows is same, so not exist duplicate data

## Missing observations

In [3]:
datafilterloan = sqlContext.sql("select loan_status,loan_amnt as lo_mnt,funded_amnt as fun_mnt,term,grade,sub_grade as s_grad,home_ownership as hom_own,verification_status as verfi_sta,purpose,title,emp_length,int_rate,annual_inc,dti,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp as TotRecPy,total_rec_int as TotRecInt,last_pymnt_amnt,last_pymnt_d from loancredit")
datafilterloan_1=sqlContext.sql("select loan_status,loan_amnt as lo_mnt,funded_amnt as fun_mnt,term,grade,sub_grade as s_grad,home_ownership as hom_own,verification_status as verfi_sta,purpose from loancredit")
sqlContext.sql("select loan_status,loan_amnt as lo_mnt,funded_amnt as fun_mnt,term,grade,sub_grade as s_grad,home_ownership as hom_own,verification_status as verfi_sta,purpose from loancredit").show(5)
datafilterloan_2=sqlContext.sql("select loan_status,title,emp_length,int_rate,annual_inc,dti,inq_last_6mths,open_acc,pub_rec,revol_bal from loancredit")
sqlContext.sql("select loan_status,title,emp_length,int_rate,annual_inc,dti,inq_last_6mths,open_acc,pub_rec,revol_bal from loancredit").show(5)
datafilterloan_3=sqlContext.sql("select loan_status,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp as TotRecPy,total_rec_int as TotRecInt,last_pymnt_amnt,last_pymnt_d from loancredit")
sqlContext.sql("select loan_status,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp as TotRecPy,total_rec_int as TotRecInt,last_pymnt_amnt,last_pymnt_d from loancredit").show(5)

+-----------+-------+-------+----------+-----+------+-------+---------------+--------------+
|loan_status| lo_mnt|fun_mnt|      term|grade|s_grad|hom_own|      verfi_sta|       purpose|
+-----------+-------+-------+----------+-----+------+-------+---------------+--------------+
| Fully Paid| 5000.0| 5000.0| 36 months|    B|    B2|   RENT|       Verified|   credit_card|
|Charged Off| 2500.0| 2500.0| 60 months|    C|    C4|   RENT|Source Verified|           car|
| Fully Paid| 2400.0| 2400.0| 36 months|    C|    C5|   RENT|   Not Verified|small_business|
| Fully Paid|10000.0|10000.0| 36 months|    C|    C1|   RENT|Source Verified|         other|
|    Current| 3000.0| 3000.0| 60 months|    B|    B5|   RENT|Source Verified|         other|
+-----------+-------+-------+----------+-----+------+-------+---------------+--------------+
only showing top 5 rows

+-----------+--------------------+----------+--------+----------+-----+--------------+--------+-------+---------+
|loan_status|           

### Discretization: Convert the categorical variable in numeric variable.
### Conversion of the following variable:
* loan_status
* term
* grade
* sub_grade
* home_ownership
* verification_status
* purpose
* title

## Convertion of loan_status

In [4]:
import pyspark.ml.feature as ft
import pyspark.sql.types as typ
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def loan_status_transformer(cat):
    if(cat == 'Fully Paid'): 
        return 1
    elif(value == 'Charged Off'): 
        return 2
    elif(value == ''): 
        return 2
    elif(value == ''): 
        return 2

dfValueToCategory = udf(CategoryToValue, IntegerType())
datafilterloan = datafilterloan.withColumn("loan_status_int", loan_status_transformer("loan_status"))


NameError: name 'CategoryToValue' is not defined

## Stratification: Put values in range.
   ### Strat the following variable:

* emp_length


Fill with mean the missing values

# Analyze data

## Descriptive Statistics

## Here data set, I am going to split in Xtrain, Y train(loan_status)
### Y train:
* loan_status
### Xtrain:
* loan_amnt  
* funded_amnt
* term
* grade
* sub_grade
* home_ownership
* verification_status
* purpose
* title: 
* emp_length
* int_rate
* annual_inc
* dti
* inq_last_6mths: information from the bureau last 6 months
* open_acc: The Number of open credit lines in the borrower's credit file.
* pub_rec: it indicates risk and hurts your ability to qualify for credit, obligations that were not paid 
* revol_bal: sometimes a full payment doesn’t (or can’t) happen, The amount of credit you have versus the amount of credit you use is a factor in your credit score
* revol_util: the amount of credit the borrower is using relative to all available revolving credit.
* total_acc: The total Number of credit lines currently in the borrower's credit file
* total_pymnt: The total payment currently
* total_pymnt_inv: The total payment currently of investment
* total_rec_prncp: 
* total_rec_int:  interest of recollect total.
* last_pymnt_amnt: amount of last payment
* last_pymnt_d: data of last payment

## Descriptive Statistic

## Outliers

In [ ]:
cols = ['loan_status','loan_amnt','funded_amnt','term','grade','sub_grade','home_ownership','verification_status','purpose','title','emp_length','int_rate','annual_inc','dti','inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util','total_acc','total_pymnt','total_pymnt_inv','total_rec_prncp as TotRecPy','total_rec_int as TotRecInt','last_pymnt_amnt','last_pymnt_d']
bounds = {}
for col in cols:
    quantiles = datafilterloan.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]

In [ ]:
outliers = datafilterloan.select(*['id'] + [
    (
        (datafilterloan[c] < bounds[c][0]) | 
        (datafilterloan[c] > bounds[c][1])
    ).alias(c + '_o') for c in cols
])
outliers.show()

In [ ]:
datafilterloan = datafilterloan.join(outliers, on='id')
datafilterloan.filter('weight_o').select('id', 'weight').show()
datafilterloan.filter('age_o').select('id', 'age').show()